### Mathematical programming

A mathematical optimization model has five components:

- Sets
- Parameters
- Decision variables
- Constraints
- Objective function(s)

In [1]:
import gurobipy as gp

In [2]:
from gurobipy import GRB

In [3]:
R = ['Carlos', 'Joe', 'Monika'] # Resources
J = ['Tester', 'JavaDeveloper', 'Architect'] # Jobs

**Overview**

The scores and combinations show the ability of one person (resource) to do the job.


If we want to maximize the score for each work where the decision variable vij=0 or 1 and sum(vji (for all i)) =1 and sum(vji (for all j)) =1, we need to add:
- sets (the combinations and scores)
- parameters=
- 

In [4]:
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

In [8]:
type(scores)

gurobipy.tupledict

In [9]:
type(combinations)

gurobipy.tuplelist

In [10]:
combinations

<gurobi.tuplelist (9 tuples, 2 values each):
 ( Carlos , Tester        )
 ( Carlos , JavaDeveloper )
 ( Carlos , Architect     )
 ( Joe    , Tester        )
 ( Joe    , JavaDeveloper )
 ( Joe    , Architect     )
 ( Monika , Tester        )
 ( Monika , JavaDeveloper )
 ( Monika , Architect     )
>

In [11]:
scores

{('Carlos', 'Tester'): 53,
 ('Carlos', 'JavaDeveloper'): 27,
 ('Carlos', 'Architect'): 13,
 ('Joe', 'Tester'): 80,
 ('Joe', 'JavaDeveloper'): 47,
 ('Joe', 'Architect'): 67,
 ('Monika', 'Tester'): 53,
 ('Monika', 'JavaDeveloper'): 73,
 ('Monika', 'Architect'): 47}

**Initialize empty model**

In [7]:
m = gp.Model('RAP')
m

<gurobi.Model Continuous instance RAP: 0 constrs, 0 vars, No parameter changes>

**Add the decision variables**

In [12]:
x = m.addVars(combinations, name="assign")

In [13]:
x

{('Carlos', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Carlos', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Joe', 'Architect'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Tester'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'JavaDeveloper'): <gurobi.Var *Awaiting Model Update*>,
 ('Monika', 'Architect'): <gurobi.Var *Awaiting Model Update*>}

**Add constraints**

In [14]:
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job') # The '*' is kindoff the same as :

In [15]:
jobs

{'Tester': <gurobi.Constr *Awaiting Model Update*>,
 'JavaDeveloper': <gurobi.Constr *Awaiting Model Update*>,
 'Architect': <gurobi.Constr *Awaiting Model Update*>}

In [18]:
resources = m.addConstrs((x.sum(r,'*') == 1 for r in R), name='resource') # The '*' is kindoff the same as :

In [19]:
resources

{'Carlos': <gurobi.Constr *Awaiting Model Update*>,
 'Joe': <gurobi.Constr *Awaiting Model Update*>,
 'Monika': <gurobi.Constr *Awaiting Model Update*>}

**Add objective**

In [20]:
m.setObjective(x.prod(scores),GRB.MAXIMIZE)

In [21]:
m

<gurobi.Model Continuous instance RAP: 0 constrs, 0 vars, No parameter changes>

In [22]:
m.write('RAP.lp')

In [23]:
m

<gurobi.Model Continuous instance RAP: 6 constrs, 9 vars, No parameter changes>

**Optimize**

In [24]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x4c93045b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       4    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [26]:
m.getVars()

[<gurobi.Var assign[Carlos,Tester] (value 1.0)>,
 <gurobi.Var assign[Carlos,JavaDeveloper] (value 0.0)>,
 <gurobi.Var assign[Carlos,Architect] (value 0.0)>,
 <gurobi.Var assign[Joe,Tester] (value 0.0)>,
 <gurobi.Var assign[Joe,JavaDeveloper] (value 0.0)>,
 <gurobi.Var assign[Joe,Architect] (value 1.0)>,
 <gurobi.Var assign[Monika,Tester] (value 0.0)>,
 <gurobi.Var assign[Monika,JavaDeveloper] (value 1.0)>,
 <gurobi.Var assign[Monika,Architect] (value 0.0)>]

In [27]:
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0


In [36]:
job_ex = m.getConstrs()[0]
job_ex

<gurobi.Constr job[Tester]>

## Changing problem - giving a total number of jobs

In [50]:
R = ['Carlos', 'Joe', 'Monika'] # Resources
J = ['Tester', 'JavaDeveloper', 'Architect'] # Jobs

**Overview**

The scores and combinations show the ability of one person (resource) to do the job.


If we want to maximize the score for each work where the decision variable vij=0 or 1 and sum(vji (for all i)) =1 and sum(vji (for all j)) =1, we need to add:
- sets (the combinations and scores)
- parameters=
- 

In [122]:
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

In [123]:
type(scores)

gurobipy.tupledict

In [124]:
type(combinations)

gurobipy.tuplelist

In [125]:
combinations

<gurobi.tuplelist (9 tuples, 2 values each):
 ( Carlos , Tester        )
 ( Carlos , JavaDeveloper )
 ( Carlos , Architect     )
 ( Joe    , Tester        )
 ( Joe    , JavaDeveloper )
 ( Joe    , Architect     )
 ( Monika , Tester        )
 ( Monika , JavaDeveloper )
 ( Monika , Architect     )
>

In [126]:
scores

{('Carlos', 'Tester'): 53,
 ('Carlos', 'JavaDeveloper'): 27,
 ('Carlos', 'Architect'): 13,
 ('Joe', 'Tester'): 80,
 ('Joe', 'JavaDeveloper'): 47,
 ('Joe', 'Architect'): 67,
 ('Monika', 'Tester'): 53,
 ('Monika', 'JavaDeveloper'): 73,
 ('Monika', 'Architect'): 47}

**Initialize empty model**

In [127]:
m = gp.Model('RAP')
m

<gurobi.Model Continuous instance RAP: 0 constrs, 0 vars, No parameter changes>

**Add Variables**

In [128]:
x = m.addVars(combinations,lb=0, name="assign")

**Add constraints**

In [129]:
J

['Tester', 'JavaDeveloper', 'Architect']

In [130]:
job1 = m.addConstr(x.sum('*',J[0]) == 20, name='job-Tester') # The '*' is kindoff the same as :

job2 = m.addConstr(x.sum('*',J[1]) == 30, name='job-JavaDeveloper') # The '*' is kindoff the same as :

job3 = m.addConstr(x.sum('*',J[2]) == 50, name='job-Architect') # The '*' is kindoff the same as :

In [131]:
R

['Carlos', 'Joe', 'Monika']

In [132]:
r1 = m.addConstr(x.sum(R[0],'*') <= 80, name='resource-Carlos') # The '*' is kindoff the same as :

r2 = m.addConstr(x.sum(R[1],'*') <= 20, name='resource-Joe') # The '*' is kindoff the same as :

r3 = m.addConstr(x.sum(R[2],'*') <= 25, name='resource-Monika') # The '*' is kindoff the same as :


**Objective**

In [133]:
m.setObjective(x.prod(scores),GRB.MAXIMIZE)

In [134]:
m.write('RAP.lp')

In [135]:
m

<gurobi.Model Continuous instance RAP: 6 constrs, 9 vars, No parameter changes>

In [136]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x386ef8a5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 8e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 8 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3720180e+03   2.751050e+01   0.000000e+00      0s
       5    4.7500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.750000000e+03


In [137]:
for v in m.getVars():
    if abs(v.x) > 1e-6:
        print(v.varName, v.x)

assign[Carlos,Tester] 20.0
assign[Carlos,JavaDeveloper] 5.0
assign[Carlos,Architect] 30.0
assign[Joe,Architect] 20.0
assign[Monika,JavaDeveloper] 25.0


#### Model to determine different hemicelluloses

In [149]:
m = gp.Model('RAP')
m

<gurobi.Model Continuous instance RAP: 0 constrs, 0 vars, No parameter changes>

In [150]:
v1 = m.addVar(lb=0)
v2 = m.addVar(lb=0)
v3 = m.addVar(lb=0)

In [151]:
m.addConstr(v1 + v2/2==80,name="Glucose")
m.addConstr(v3/2==10,name="Galactose")
m.addConstr(v2/3==5,name="Arabinose")
m.addConstr(2*v2/3 + v3/4==5,name="Xylose")

<gurobi.Constr *Awaiting Model Update*>

In [152]:
m.setObjective(v1+v2+v3,GRB.MAXIMIZE)

In [153]:
m.write('RAP.lp')

In [154]:
m

<gurobi.Model Continuous instance RAP: 4 constrs, 3 vars, No parameter changes>

In [155]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x40fdc549
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 8e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model
